# Análisis de Aspiradoras

El Objetivo de esta Note es los siguientes:

1. Obtener información de la página de Amazon sobre aspiradoras
2. Guardar las información de las 200 primeras aspiradoras en un dataframe
3. Pasar el DataFrame y aplicarle técnicas de aprendizaje No Supervisado para clasificar las aspiradoras

## Obtener Información de Aspiradoras de Amazon

### Web Scraping

#### BeautifulSoup

Es el primer método que vamos a utilizar para poder extraer datos de la Web

In [140]:
#Líbrerias
import requests

#url = 'https://www.amazon.com/'
url = 'https://www.amazon.es/s?k=aspiradoras&__mk_es_ES=%C3%85M%C3%85%C5%BD%C3%95%C3%91&ref=nb_sb_noss_2'

aspiradoraWeb = requests.get(url)
aspiradoraWeb.status_code

503

* El primer método de Scraping iba hacer con BeautifulSoup, sin embargo, tengo un inconveniente debido a que el status code me da 503, eso significa que no esta accesible el servidor, esto puede deberse a que hay páginas que no permiten realizar el WebScrapper por la líbreria requests

* Para resolver este caso vamos a utilizar Selenium

#### Selenium

##### 1. Configuracion y parametrización de Selenium

In [142]:
#Importamos Selenium
from selenium import webdriver

#Parametrizamos el driver
options = webdriver.ChromeOptions()
#Para navegar en modo incognito
options.add_argument('--incognito')

#Abrimos el driver
driver = webdriver.Chrome(executable_path='c:/chromedriver.exe',options=options)

#Nos vamos a la URL
driver.get(url)

##### 2. Obtener los links de cada una de las aspiradoras que se encuentran en una página y lo almaceno en una lista

In [143]:
# Vamos a obtener los links y los nombres de las aspiradoras
aspiradoras = driver.find_elements_by_xpath('//a[@class="a-link-normal a-text-normal"]')

#Número de aspiradoras 
len(aspiradoras)

22

In [144]:
#vamos a obtener los links de cada aspiradora
linksAspiradoras = []
aspiradora = aspiradoras[0]

linkAspiradora = aspiradora.get_attribute('href')
linkAspiradora

#Obtengo los links de las aspiradoras
for aspiradora in aspiradoras:
    linksAspiradoras.append(aspiradora.get_attribute('href'))
    
linksAspiradoras

['https://www.amazon.es/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A01147902Q6W59R82Z299&url=%2FAmazonBasics-Aspiradora-cil%25C3%25ADndrica-bolsa-energ%25C3%25A9tica%2Fdp%2FB07C3NFCYC%2Fref%3Dsr_1_1_sspa%3F__mk_es_ES%3D%25C3%2585M%25C3%2585%25C5%25BD%25C3%2595%25C3%2591%26keywords%3Daspiradoras%26qid%3D1583447383%26sr%3D8-1-spons%26psc%3D1&qualifier=1583447383&id=873035640061113&widgetName=sp_atf',
 'https://www.amazon.es/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_2?ie=UTF8&adId=A067223035FI8SQQRCGZA&url=%2FCecotec-EcoExtreme-Aspirador-Cicl%25C3%25B3nico-Capacidad%2Fdp%2FB07587LM6H%2Fref%3Dsr_1_2_sspa%3F__mk_es_ES%3D%25C3%2585M%25C3%2585%25C5%25BD%25C3%2595%25C3%2591%26keywords%3Daspiradoras%26qid%3D1583447383%26sr%3D8-2-spons%26psc%3D1&qualifier=1583447383&id=873035640061113&widgetName=sp_atf',
 'https://www.amazon.es/Rowenta-Compact-Power-Cyclonic-Aspirador/dp/B01CIALC5G/ref=sr_1_3?__mk_es_ES=%C3%85M%C3%85%C5%BD%C3%95%C3%91&keywords=as

In [145]:
# Se cierra el driver
driver.close()

##### 3. Vamos a obtener la información de cada aspiradora:

    * Nombre
    * Precio
    * Número de Opiniones
    * Tabla de detalles técnicos

In [146]:
# Vamos a obtener los datos de cada aspiradora
driver = webdriver.Chrome(executable_path='c:/chromedriver.exe',options=options)
# Obtengo el primer link del links de aspiradoras
driver.get(linksAspiradoras[0])

In [147]:
#Se obtiene el nombre de la aspiradora
nombreAspiradora = driver.find_element_by_xpath('//*[@id="productTitle"]').text
#Se obtiene el número de opiniones
numeroOpinion = driver.find_element_by_xpath('//*[@id="acrCustomerReviewText"]').text
#Se obtiene el precio
precio = driver.find_element_by_xpath('//*[@id="priceblock_ourprice"]').text

print(nombreAspiradora)
print(numeroOpinion)
print(precio)

Holife Aspiradora Escoba de 15Kpa, Aspirador Vertical 2 en 1, 800ml, con 2 Filtros HEPA Lavables y 3 Cepillos, Vertical y Portátil Ajustable [Clase de eficiencia energética A+] (Blanco1)
37 valoraciones
41,98 €


In [148]:
#Obteniendo la tabla de caracteristicas técnicas
detallesTecnicos = driver.find_elements_by_xpath('//*[@id="prodDetails"]/div[2]/div[1]/div[1]/div[2]/div/div/table/tbody/tr')
detallesTecnicos


[<selenium.webdriver.remote.webelement.WebElement (session="eb70002b8fe8a2125609793597fbde2b", element="10cf63cb-30e6-4e25-bfe0-d4be53cb6345")>,
 <selenium.webdriver.remote.webelement.WebElement (session="eb70002b8fe8a2125609793597fbde2b", element="39541a98-b1bb-4c93-80e0-c7f856b06634")>,
 <selenium.webdriver.remote.webelement.WebElement (session="eb70002b8fe8a2125609793597fbde2b", element="d3115e33-611c-41fd-8a9c-2414e261df43")>,
 <selenium.webdriver.remote.webelement.WebElement (session="eb70002b8fe8a2125609793597fbde2b", element="2a693ad1-3b9b-4ff1-bcd6-dbc0877faf9c")>,
 <selenium.webdriver.remote.webelement.WebElement (session="eb70002b8fe8a2125609793597fbde2b", element="5ac00d4e-0139-45ea-8e46-601ec6d6d0d4")>,
 <selenium.webdriver.remote.webelement.WebElement (session="eb70002b8fe8a2125609793597fbde2b", element="f6684a3d-98a9-4f3e-bc2d-4e8097305fb0")>,
 <selenium.webdriver.remote.webelement.WebElement (session="eb70002b8fe8a2125609793597fbde2b", element="abc03fa2-82c9-41b6-88c2-65

In [149]:
# Obtengo el primer detalle técnico de la table en este caso la etiqueta
detallesTecnicos[0].find_element_by_xpath('.//td[@class="label"]').text
# Obtengo el primer detalle técnico de la table en este caso el valor
detallesTecnicos[0].find_element_by_xpath('.//td[@class="value"]').text

tablaDetalleTecnico = {}

for detalle in detallesTecnicos:
    try:
        etiqueta = detalle.find_element_by_xpath('.//td[@class="label"]').text
        valor = detalle.find_element_by_xpath('.//td[@class="value"]').text
    
        # El valor le paso como una lista para que sea mas fácil la conversion a Dataframe
        tablaDetalleTecnico.update({etiqueta:[valor]})
    except:
        etiqueta = ''
        valor = ''    
        
    
tablaDetalleTecnico

{'Marca': ['Holife'],
 'Color': ['Blanco1'],
 'Peso del producto': ['2,16 Kg'],
 'Dimensiones del producto': ['14 x 15,5 x 64 cm'],
 'Capacidad': ['0.8 litros'],
 'Volumen': ['0.8 litros'],
 'Potencia': ['600 vatios'],
 'Ruido': ['84 dB']}

##### 4. Pasar a un Dataframe los datos

In [150]:
#Mandando a un dataframe los detalles tecnicos

import pandas as pd

tablaDetalleTecnico

DatoAspiradora = pd.DataFrame(tablaDetalleTecnico)
DatoAspiradora

,Marca,Color,Peso del producto,Dimensiones del producto,Capacidad,Volumen,Potencia,Ruido
0,Holife,Blanco1,"2,16 Kg","14 x 15,5 x 64 cm",0.8 litros,0.8 litros,600 vatios,84 dB


In [151]:
print(nombreAspiradora)
print(numeroOpinion)
print(precio)

Holife Aspiradora Escoba de 15Kpa, Aspirador Vertical 2 en 1, 800ml, con 2 Filtros HEPA Lavables y 3 Cepillos, Vertical y Portátil Ajustable [Clase de eficiencia energética A+] (Blanco1)
37 valoraciones
41,98 €


In [152]:
# Vamos añadir las columnas de:
# Nombre de la aspiradora; las valoraciones; y el precio en el DataFrame
DatoAspiradora['Nombre Aspiradora'] = nombreAspiradora
DatoAspiradora['Opiniones'] = numeroOpinion
DatoAspiradora['Precio'] = precio

DatoAspiradora

,Marca,Color,Peso del producto,Dimensiones del producto,Capacidad,Volumen,Potencia,Ruido,Nombre Aspiradora,Opiniones,Precio
0,Holife,Blanco1,"2,16 Kg","14 x 15,5 x 64 cm",0.8 litros,0.8 litros,600 vatios,84 dB,"Holife Aspiradora Escoba de 15Kpa, Aspirador V...",37 valoraciones,"41,98 €"


##### 5. Obtenemos solo los datos que necesito para el Análisis

    * Nombre de la aspiradora
    * Precio
    * Número de Opiniones
    * Peso del producto
    * Potencia
    * Dimensiones del producto
    * Volúmen

In [153]:
#Veo las columnas que tiene el Dataframe
DatoAspiradora.columns

Index(['Marca', 'Color', 'Peso del producto', 'Dimensiones del producto',
       'Capacidad', 'Volumen', 'Potencia', 'Ruido', 'Nombre Aspiradora',
       'Opiniones', 'Precio'],
      dtype='object')

In [154]:
# Voy a obtener unicamente los valores que necesito

resultadoAspiradoras = DatoAspiradora[['Nombre Aspiradora','Precio','Opiniones','Peso del producto','Potencia'
                                       ,'Dimensiones del producto','Volumen']]

resultadoAspiradoras

,Nombre Aspiradora,Precio,Opiniones,Peso del producto,Potencia,Dimensiones del producto,Volumen
0,"Holife Aspiradora Escoba de 15Kpa, Aspirador V...","41,98 €",37 valoraciones,"2,16 Kg",600 vatios,"14 x 15,5 x 64 cm",0.8 litros


##### 6. Obtener todos los datos de las 200 aspiradoras